<a href="https://colab.research.google.com/github/amandaliusa/cs155-kaggle/blob/master/catboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import roc_curve, auc

In [0]:
import tensorflow as tf
from tensorflow.python.data import Dataset
import keras
from keras.utils import to_categorical
from keras import models
from keras import layers

In [2]:
from google.colab import files
uploaded = files.upload()

Saving test.csv to test.csv
Saving train.csv to train.csv


In [0]:
import io
df_train = pd.read_csv(io.BytesIO(uploaded['train.csv']))
df_test = pd.read_csv(io.BytesIO(uploaded['test.csv'] ))

In [0]:
# replace missing values with median (less sensitive to outliers)
df_train['opened_position_qty '].fillna(df_train['opened_position_qty '].median(),inplace=True)
df_test['opened_position_qty '].fillna(df_train['opened_position_qty '].median(),inplace=True)
df_train['closed_position_qty'].fillna(df_train['closed_position_qty'].median(),inplace=True)
df_test['closed_position_qty'].fillna(df_train['closed_position_qty'].median(), inplace=True)

In [0]:
df = pd.concat([df_train,df_test],sort=False)

In [0]:
# engineer new features by grouping like features and computing statistics on them 
bids = ['bid1','bid2', 'bid3', 'bid4', 'bid5']
bid_vols = ['bid1vol', 'bid2vol', 'bid3vol', 'bid4vol', 'bid5vol']
asks = ['ask1', 'ask2', 'ask3', 'ask4', 'ask5',]
ask_vols = ['ask1vol','ask2vol', 'ask3vol', 'ask4vol', 'ask5vol']
mid2 = ['bid2', 'ask2']
mid3 = ['bid3', 'ask3']
mid4 = ['bid4', 'ask4']
mid5 = ['bid5', 'ask5']

group_cols = {"bids":bids,"bid_vols":bid_vols,"asks":asks,"ask_vols":ask_vols}
group_cols2 = {"mid2":mid2, "mid3":mid3, "mid4":mid4, "mid5":mid5}

In [30]:
for group in group_cols.keys():
    print(group)
    df[f"{group}_max"] = df[group_cols[group]].max(axis=1)
    df[f"{group}_avg"] = df[group_cols[group]].mean(axis=1)
    df[f"{group}_std"] = df[group_cols[group]].std(axis=1)
    df[f"{group}_min"] = df[group_cols[group]].min(axis=1)
    df[f"{group}_spread"] = df[f"{group}_max"].div(df[f"{group}_min"])
  
for group in group_cols2.keys():
    # take average of bid/ask prices for 2,3,4,5
    print(group)
    df[f"{group}"] = df[group_cols2[group]].mean(axis=1)
      
df["last_price/mid"] = df["last_price"].div(df["mid"])

bids
bid_vols
asks
ask_vols
mid2
mid3
mid4
mid5


In [0]:
x_train = df.loc[~df.y.isna()]
y_train = x_train['y']
x_train = x_train.drop(['y'], axis=1)
x_test = df.loc[df.y.isna()]
x_test = x_test.drop(['y'], axis=1)

In [0]:
# Normalize data
from sklearn import preprocessing

# Normalize training data by subtracting mean and scaling to unit variance
std_scale = preprocessing.StandardScaler().fit(x_train)
x_train_norm = std_scale.transform(x_train)
x_train = pd.DataFrame(x_train_norm, index=x_train.index, columns=x_train.columns)

# Normalize testing data by using mean and SD of training set
x_test_norm = std_scale.transform(x_test)
x_test = pd.DataFrame(x_test_norm, index=x_test.index, columns=x_test.columns) 

In [0]:
# Split training and validation data 
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2)

In [11]:
pip install catboost

     |████████████████████████████████| 64.0MB 46kB/s 


In [33]:
from catboost import CatBoostClassifier
model = CatBoostClassifier(
    iterations=10000,
    learning_rate=0.1,
    task_type='GPU'
)
  
model.fit(
      x_train, y_train,
      #eval_set=(x_val, y_val),
      verbose=100,
      use_best_model=True
)  

print(model.get_best_score())


You should provide test set for use best model. use_best_model parameter has been switched to false value.


0:	learn: 0.6788588	total: 27ms	remaining: 4m 30s
100:	learn: 0.6102838	total: 1.39s	remaining: 2m 15s
200:	learn: 0.6079310	total: 2.69s	remaining: 2m 10s
300:	learn: 0.6059910	total: 4s	remaining: 2m 8s
400:	learn: 0.6043267	total: 5.29s	remaining: 2m 6s
500:	learn: 0.6027951	total: 6.44s	remaining: 2m 2s
600:	learn: 0.6013520	total: 7.68s	remaining: 2m
700:	learn: 0.6000179	total: 8.83s	remaining: 1m 57s
800:	learn: 0.5986962	total: 9.97s	remaining: 1m 54s
900:	learn: 0.5973573	total: 11.1s	remaining: 1m 52s
1000:	learn: 0.5960479	total: 12.3s	remaining: 1m 50s
1100:	learn: 0.5947968	total: 13.5s	remaining: 1m 49s
1200:	learn: 0.5935710	total: 14.8s	remaining: 1m 48s
1300:	learn: 0.5923826	total: 16.1s	remaining: 1m 47s
1400:	learn: 0.5912032	total: 17.4s	remaining: 1m 46s
1500:	learn: 0.5900797	total: 18.8s	remaining: 1m 46s
1600:	learn: 0.5889528	total: 20.1s	remaining: 1m 45s
1700:	learn: 0.5878348	total: 21.5s	remaining: 1m 44s
1800:	learn: 0.5867218	total: 22.8s	remaining: 1m 4

In [13]:
y_pred = model.predict(x_val)
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_val, y_pred)
roc_auc = auc(false_positive_rate, true_positive_rate)
print('val auc: {}'.format(roc_auc))

val auc: 0.588710174132235


In [17]:
from google.colab import drive
drive.mount('/content/gdrive')
#using model.save_weights() to save the weights of the model in HDF5 format
df_test['Predicted'] = model.predict_proba(x_test)[:,1]
df_test[["id","Predicted"]].to_csv('/content/gdrive/My Drive/submission_cat.csv', index=False)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
df_test['Predicted'] = model.predict_proba(x_test)[:,1]
df_test[["id","Predicted"]].to_csv('/content/gdrive/My Drive/submission_cat2.csv', index=False)

In [17]:
from sklearn.ensemble import RandomForestClassifier
model4 = RandomForestClassifier(n_estimators=64)
model4.fit(x_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=64,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [34]:
from sklearn.ensemble import AdaBoostClassifier

boosting3 = AdaBoostClassifier(base_estimator=model, n_estimators=2)   
boosting3.fit(x_train, y_train)
print(boosting3.score(x_train, y_train))
print(boosting3.score(x_val, y_val))

Streaming output truncated to the last 5000 lines.
5003:	learn: 0.6915126	total: 1m 3s	remaining: 1m 3s
5004:	learn: 0.6915125	total: 1m 3s	remaining: 1m 3s
5005:	learn: 0.6915124	total: 1m 3s	remaining: 1m 3s
5006:	learn: 0.6915122	total: 1m 3s	remaining: 1m 3s
5007:	learn: 0.6915123	total: 1m 3s	remaining: 1m 3s
5008:	learn: 0.6915121	total: 1m 3s	remaining: 1m 3s
5009:	learn: 0.6915119	total: 1m 3s	remaining: 1m 3s
5010:	learn: 0.6915119	total: 1m 3s	remaining: 1m 3s
5011:	learn: 0.6915109	total: 1m 3s	remaining: 1m 3s
5012:	learn: 0.6915106	total: 1m 3s	remaining: 1m 3s
5013:	learn: 0.6915105	total: 1m 3s	remaining: 1m 3s
5014:	learn: 0.6915102	total: 1m 3s	remaining: 1m 3s
5015:	learn: 0.6915100	total: 1m 3s	remaining: 1m 3s
5016:	learn: 0.6915101	total: 1m 3s	remaining: 1m 3s
5017:	learn: 0.6915101	total: 1m 3s	remaining: 1m 3s
5018:	learn: 0.6915100	total: 1m 3s	remaining: 1m 3s
5019:	learn: 0.6915094	total: 1m 3s	remaining: 1m 3s
5020:	learn: 0.6915090	total: 1m 3s	remaining: 1

In [35]:
y_pred = boosting3.predict(x_val)
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_val, y_pred)
roc_auc = auc(false_positive_rate, true_positive_rate)
print('val auc: {}'.format(roc_auc))

val auc: 0.5736709966740402


In [0]:
df_test['Predicted'] = boosting3.predict_proba(x_test)[:,1]
df_test[["id","Predicted"]].to_csv('/content/gdrive/My Drive/submission_cat11.csv', index=False)